# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-unicos-filtrados-6000.csv', sep = '|')
df['filtrado_6000']=df['filtrado_6000'].astype(str)
df.head()

,acordao,areas,filtrado_6000
0,297/2016-P,Responsabilidade,tribunal conta união tc grupo classe plenário ...
1,366/2016-P,Finanças Públicas,tribunal conta união tc grupo classe ii plenár...
2,944/2016-P,Responsabilidade,tribunal conta união tc grupo classe plenário ...
3,30/2016-P,Direito Processual,tribunal conta união tc grupo classe plenário ...
4,55/2016-P,Pessoal,wania lucia pasquarelli nascimentotcuwania luc...


In [2]:
df.shape

(9739, 3)

In [3]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['areas']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [4]:
y = lbArea.transform(df['areas'])
y.shape

(9739, 10)

In [5]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 100000
limite_texto = 6000
dim_vetor = 50

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['filtrado_6000'])

sequences = tokenizer.texts_to_sequences(df['filtrado_6000'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 95423 unique tokens.


In [6]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (9739, 6000)


In [7]:
x.shape, y.shape

((9739, 6000), (9739, 10))

# Treinamento

In [8]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense, Bidirectional
from keras.layers.core import Dropout
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0312 21:59:15.919392 139780508272448 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0312 21:59:15.933649 139780508272448 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0312 21:59:15.935350 139780508272448 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0312 21:59:16.252184 139780508272448 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 6000, 50)          5000000   
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 5,238,346
Trainable params: 5,238,346
Non-trainable params: 0
_________________________________________________________________


W0312 21:59:16.568278 139780508272448 deprecation.py:323] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0312 21:59:16.978687 139780508272448 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 7791 samples, validate on 1948 samples
Epoch 1/20
7791/7791 [==============================] - 1179s 151ms/step - loss: 1.8306 - categorical_accuracy: 0.3729 - val_loss: 1.9571 - val_categorical_accuracy: 0.2125
Epoch 2/20
7791/7791 [==============================] - 1185s 152ms/step - loss: 1.7547 - categorical_accuracy: 0.4007 - val_loss: 1.9579 - val_categorical_accuracy: 0.2161
Epoch 3/20
7791/7791 [==============================] - 1178s 151ms/step - loss: 1.6487 - categorical_accuracy: 0.4401 - val_loss: 1.5829 - val_categorical_accuracy: 0.3706
Epoch 4/20
7791/7791 [==============================] - 1194s 153ms/step - loss: 1.3879 - categorical_accuracy: 0.5238 - val_loss: 1.5167 - val_categorical_accuracy: 0.4574
Epoch 5/20
7791/7791 [==============================] - 1183s 152ms/step - loss: 1.2172 - categorical_accuracy: 0.5915 - val_loss: 1.3815 - val_categorical_accuracy: 0.5323
Epoch 6/20
7791/7791 [==============================] - 1185s 152ms/step - loss: 1.0981

In [9]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0313 04:33:16.681062 139780508272448 deprecation.py:506] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 6000, 50)          5000000   
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 5,238,346
Trainable params: 5,238,346
Non-trainable params: 0
_________________________________________________________________
Train on 7791 samples, validate on 1948 samples
Epoch 1/20
7791/7791 [==============================] - 1461s 188ms/step - loss: 1.8269 - categorical_accuracy: 0.3625 - val_loss: 1.9485 - val_categorical_accuracy: 0.2151
Epoch 2/20
7791/7791 [==============================] - 1459s 187ms/step - loss: 1.7475 - categorical_accuracy: 0.4023 - val_loss: 1.8896 - val_categorical_accuracy: 0.2166
Ep

In [10]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 6000, 50)          5000000   
_________________________________________________________________
gru_3 (GRU)                  (None, 512)               864768    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 5,869,898
Trainable params: 5,869,898
Non-trainable params: 0
_________________________________________________________________
Train on 7791 samples, validate on 1948 samples
Epoch 1/20
7791/7791 [==============================] - 1515s 195ms/step - loss: 1.8466 - categorical_accuracy: 0.3672 - val_loss: 1.9189 - val_categorical_accuracy: 0.2125
Epoch 2/20
7791/7791 [==============================] - 1511s 194ms/step - loss: 0.9046 - categorical_accuracy: 0.2285 - val_loss: 1.1921e-07 - val_categorical_accuracy: 0.035

In [11]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 6000, 50)          5000000   
_________________________________________________________________
gru_4 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
Total params: 5,238,346
Trainable params: 5,238,346
Non-trainable params: 0
_________________________________________________________________
Train on 7791 samples, validate on 1948 samples
Epoch 1/20
7791/7791 [==============================] - 1489s 191ms/step - loss: 1.8537 - categorical_accuracy: 0.3605 - val_loss: 1.8935 - val_categorical_accuracy: 0.2151
Epoch 2/20
7791/7791 [==============================] - 1487s 191ms/step - loss: 1.7461 - categorical_accuracy: 0.4023 - val_loss: 1.9183 - val_categorical_accuracy: 0.2166
Ep

In [12]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Bidirectional(GRU(256, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 6000, 50)          5000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               471552    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 5,476,682
Trainable params: 5,476,682
Non-trainable params: 0
_________________________________________________________________
Train on 7791 samples, validate on 1948 samples
Epoch 1/20
7791/7791 [==============================] - 3635s 467ms/step - loss: 1.8341 - categorical_accuracy: 0.3744 - val_loss: 1.8992 - val_categorical_accuracy: 0.2084
Epoch 2/20
7791/7791 [==============================] - 3529s 453ms/step - loss: 1.7530 - categorical_accuracy: 0.4051 - val_loss: 1.8469 - val_categorical_accuracy: 0.2259
Ep

KeyboardInterrupt: 